In [1]:
from datasets import load_dataset
import polars as pl
dataset = load_dataset("microsoft/ms_marco", "v1.1")

data_dict = dataset["train"].to_dict()  # Convert to a dictionary of columns
            
            # Convert the dictionary to a Polars DataFrame
df = pl.DataFrame(data_dict)

/home/tomer/dynamic-rechunking-RAG/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm
# Initialize the model
model = SentenceTransformer('intfloat/e5-base-v2')

# Initialize Pinecone
pinecone_index_name="dynamic"
pinecone_api_key="pcsk_5SLo1E_SPpKBieD8wQRuRf9G7xfYrVWDLWUAkfRC3X5xjMpDhy7j8CH3SEen8kJiNbMjav"
pc=Pinecone(api_key=pinecone_api_key)

# Create Pinecone index if it doesn't exist
print("Creating a Pinecone index...")
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if pinecone_index_name not in existing_indexes:
    pc.create_index(
        name=pinecone_index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
pinecone_index=pc.Index(pinecone_index_name)

index=pc.Index(pinecone_index_name)

batch_size=100

to_upsert = []
for i in tqdm(range(0, len(df), batch_size)):
# Extract the batch of rows
    batch = df.slice(i, batch_size)

    # Extract passage texts and prepare metadata
    texts_to_embed = []
    metadata_list = []
    for row_idx, item in enumerate(batch.select("passages").to_numpy().flatten()):
        print(item)

        if isinstance(item, list) and len(item) > 1:
            # Extract passages from the second element of the list
            passage_texts = item[1]
            if isinstance(passage_texts, list):
                for passage in passage_texts:
                    texts_to_embed.append(f"passage: {passage}")
                    metadata_list.append({"row_number": i + row_idx})  # Same row index for all passages
            else:
                print(f"Skipping row {i + row_idx}: 'passage_texts' is not a list.")
        else:
            print(f"Skipping row {i + row_idx}: Invalid 'passages' structure.")

    # Generate embeddings for all passages in the batch
    embeddings = model.encode(texts_to_embed, convert_to_numpy=True, batch_size=batch_size)

    # Prepare list of (id, vector, metadata) dictionaries for upsert
    for passage_idx, (embedding, metadata) in enumerate(zip(embeddings, metadata_list)):
        # Unique ID includes row number and passage index
        doc_id = f"row-{metadata['row_number']}-passage-{passage_idx}"
        to_upsert.append({
            "id": doc_id,
            "values": embedding.tolist(),
            "metadata": metadata
        })

    # Upsert in Pinecone in batches
    if len(to_upsert) >= batch_size:
        pinecone_index.upsert(vectors=to_upsert)
        to_upsert = []  # Reset the upsert list

# Handle any remaining vectors
if to_upsert:
    pinecone_index.upsert(vectors=to_upsert)

print("Dense indexing completed.")


10